## E. Batbaatar, M. Li, and K. H. Ryu, “Semantic-emotion neural network for emotion recognition from text,” IEEE access, vol. 7, pp. 111 866–111 878, 2019.

In [ ]:
import os
import matplotlib.pyplot as plt
import re
import string
import json
import emoji
import numpy as np
import pandas as pd
from sklearn import metrics
from bs4 import BeautifulSoup
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
import warnings

In [ ]:
import torch
print(torch.cuda.is_available())


False


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path1 = '/content/drive/My Drive/PROJECT/Text Data/train.tsv'
path2 = '/content/drive/My Drive/PROJECT/Text Data/dev.tsv'


In [ ]:
import pandas as pd

df_train = pd.read_csv(path1, sep='\t', header=None, names=['Text', 'Class', 'ID'])
df_dev = pd.read_csv(path2, sep='\t', header=None, names=['Text', 'Class', 'ID'])

In [ ]:
df_train

,Text,Class,ID
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj
...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,18,edsb738
43406,Always thought that was funny but is it a refe...,6,ee7fdou
43407,What are you talking about? Anything bad that ...,3,efgbhks
43408,"More like a baptism, with sexy results!",13,ed1naf8


In [ ]:
df_train['List of classes'] = df_train['Class'].apply(lambda x: x.split(','))
df_train['Len of classes'] = df_train['List of classes'].apply(lambda x: len(x))
df_dev['List of classes'] = df_dev['Class'].apply(lambda x: x.split(','))
df_dev['Len of classes'] = df_dev['List of classes'].apply(lambda x: len(x))

In [ ]:
MAPS = 128
noise_dim = 128

from tensorflow.keras import layers
from tensorflow.keras.initializers import RandomNormal
init = RandomNormal(mean=0.0, stddev=0.02)

def make_generator():
    seed = tf.keras.Input(shape=((noise_dim,)))
    label = tf.keras.Input(shape=((1,)))
    x = layers.Embedding(120, 120, input_length=1,name='emb')(label)
    x = layers.Flatten()(x)
    x = layers.concatenate([seed,x])
    x = layers.Dense(4*4*MAPS*8, use_bias=False)(x)
    x = layers.Reshape((4, 4, MAPS*8))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(MAPS*4, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(MAPS*2, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(MAPS, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False, activation='tanh')(x)

    model = tf.keras.Model(inputs=[seed,label], outputs=x)
    return model

generator = make_generator()
#generator.summary()

In [ ]:
GNOISE = 0.25

def make_discriminator():
    image = tf.keras.Input(shape=((64,64,3)))
    label = tf.keras.Input(shape=((1,)))
    x = layers.Embedding(120, 64*64, input_length=1)(label)
    x = layers.Reshape((64,64,1))(x)
    x = layers.concatenate([image,x])

    x = layers.Conv2D(MAPS, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    #x = layers.GaussianNoise(GNOISE)(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(MAPS*2, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    #x = layers.GaussianNoise(GNOISE)(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(MAPS*4, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    #x = layers.GaussianNoise(GNOISE)(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(MAPS*8, (5, 5), strides=(2, 2), padding='same', kernel_initializer=init, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    #x = layers.GaussianNoise(GNOISE)(x)
    x = layers.LeakyReLU()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(121, activation='sigmoid')(x)
    x2 = layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inputs=[image,label], outputs=[x,x2])
    return model

discriminator = make_discriminator()

In [ ]:
# THESE LOSS FUNCTIONS ARE UNUSED. THEY ARE REWRITTEN INSIDE TRAINING LOOP BELOW
from tensorflow.contrib.eager.python import tfe

# RaLS Discriminator Loss
def RaLS_errD(fake,real):
    return (tf.reduce_mean( (real - tf.reduce_mean(fake,0) - tf.ones_like(real))**2,0 )
        + tf.reduce_mean( (fake - tf.reduce_mean(real,0) + tf.ones_like(real))**2,0 ) )/2.

# RaLS Generator Loss
def RaLS_errG(fake,real):
    return (tf.reduce_mean( (real - tf.reduce_mean(fake,0) + tf.ones_like(real))**2,0 )
        + tf.reduce_mean( (fake - tf.reduce_mean(real,0) - tf.ones_like(real))**2,0 ) )/2.

# OPTIMIZER - ADAM
learning_rate = tfe.Variable(0.0002)
generator_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
discriminator_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)

In [3]:
EPOCHS = 25
num_examples = 16

@ tf.function
def train_step(images,generator,discriminator,generator_optimizer,discriminator_optimizer):

    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True,label_smoothing=0.4)
    bce2 = tf.keras.losses.BinaryCrossentropy(from_logits=False,label_smoothing=0.4)
    noise = tf.random.normal((32,128)) # update noise_dim here
    labs = tf.cast(120*tf.random.uniform((32,)),tf.int32)

    # USE GRADIENT TAPE TO CALCULATE GRADIENTS
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator([noise,labs], training=True)
        real_cat, real_output = discriminator([images[0],images[1]], training=True)
        fake_cat, fake_output = discriminator([generated_images,labs], training=True)

        # GENERATOR LOSS
        gen_loss = (tf.reduce_mean( (real_output - tf.reduce_mean(fake_output,0) + tf.ones_like(real_output))**2,0 )
        + tf.reduce_mean( (fake_output - tf.reduce_mean(real_output,0) - tf.ones_like(real_output))**2,0 ) )/2.

        # DISCRIMINATOR LOSS
        disc_loss = bce(tf.ones_like(real_output), real_output) + bce(tf.zeros_like(fake_output), fake_output)
        real_cat2 = tf.one_hot(tf.cast(images[1],tf.int32),121,dtype=tf.int32)
        fake_cat2 = tf.one_hot(120*tf.ones((32,),tf.int32),121,dtype=tf.int32)
        disc_loss += bce2(real_cat2,real_cat) + bce2(fake_cat2,fake_cat)

    # BACK PROPAGATE ERROR
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

print('Training started. Displaying every '+str(DISPLAY_EVERY)+'th epoch.')
train(ds, EPOCHS)

Epoch 1/9
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 118s 36ms/step - accuracy: 0.1909 - loss: 0.9941 - val_accuracy: 0.1258 - val_loss: 1.1877

Epoch 2/9
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 108s 34ms/step - accuracy: 0.2989 - loss: 0.7947 - val_accuracy: 0.2104 - val_loss: 0.9491

Epoch 3/9
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 112s 36ms/step - accuracy: 0.4089 - loss: 0.7655 - val_accuracy: 0.3295 - val_loss: 1.0838

Epoch 4/9
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 109s 35ms/step - accuracy: 0.4246 - loss: 0.7240 - val_accuracy: 0.4898 - val_loss: 1.4297

Epoch 5/9
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 144s 35ms/step - accuracy: 0.5327 - loss: 0.7059 - val_accuracy: 0.4868 - val_loss: 1.1048

Epoch 6/9
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 138s 34ms/step - accuracy: 0.5465 - loss: 0.6769 - val_accuracy: 0.5390 - val_loss: 1.5040

Epoch 7/9
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 106s 34ms/step - accuracy: 0.7271 - loss: 0.6451 - val_accuracy: 0.7072 - val_loss: 1.0873

Epoch 8/9
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 144s 35ms/step - accuracy: 0